<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/110_Claude_Code_Clean_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Claude Code Starter Notebook

This notebook is a clean template for working with **Claude** (Anthropic's models) in Colab.

It supports:
- Loading your API key from a `.env` file
- A helper function `ask_claude` for single-turn Q&A
- A simple **conversation manager** to keep history across multiple turns
- Running shell commands via `!` or `%%bash`


## 1. Install dependencies

In [1]:

!pip -q install anthropic python-dotenv rich


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 9.3 MB/s eta 0:00:00


## 2. Load API key

In [2]:

import os
from dotenv import load_dotenv

# Adjust path to your secrets file
load_dotenv("/content/API_KEYS.env")

anthropic_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY in /content/API_KEYS.env")

print("✅ Anthropic key loaded")


✅ Anthropic key loaded


## 3. Import libraries and set up client

In [3]:

from anthropic import Anthropic, APIError
from rich.console import Console
from rich.markdown import Markdown

console = Console()
client = Anthropic(api_key=anthropic_key)

# Default to Claude 3.5 Haiku for speed & low cost
MODEL_NAME = os.environ.get("CLAUDE_MODEL", "claude-3-5-haiku-latest")


## 4. Helper function for single-turn queries

In [16]:
import textwrap

def smart_print_markdown(output: str, width: int = 100):
    """Wrap plain text, preserve code fences."""
    in_code = False
    buf = []
    for line in output.splitlines():
        if line.strip().startswith("```"):
            # flush any wrapped text before toggling code mode
            if buf:
                print(textwrap.fill(" ".join(buf), width=width, replace_whitespace=False))
                print()
                buf = []
            print(line)
            in_code = not in_code
            continue
        if in_code:
            print(line)
        else:
            # collect non-code lines to wrap as paragraphs
            if line.strip() == "":
                if buf:
                    print(textwrap.fill(" ".join(buf), width=width, replace_whitespace=False))
                    print()
                    buf = []
            else:
                buf.append(line)
    if buf:
        print(textwrap.fill(" ".join(buf), width=width, replace_whitespace=False))
        print()

def ask_claude(prompt: str, system: str = "You are a helpful coding assistant.",
               render: str = "markdown",  # 'markdown' | 'wrapped' | 'none'
               return_text: bool = False) -> str | None:
    if not anthropic_key:
        raise RuntimeError("Missing ANTHROPIC_API_KEY.")
    msg = client.messages.create(
        model=MODEL_NAME,
        max_tokens=1000,
        temperature=0.2,
        system=system,
        messages=[{"role": "user", "content": prompt}],
    )
    parts = [b.text for b in msg.content if getattr(b, "type", None) == "text"]
    output = "\n\n".join(parts).strip() or "(No text)"

    if render == "markdown":
        console.print(Markdown(output))
    elif render == "wrapped":
        smart_print_markdown(output)
    # render == 'none' skips printing

    return output if return_text else None


In [17]:
# Single-turn example
ask_claude("Write a Python function that reverses a string.")

Here's a Python function that reverses a string:                                                                   

                                                                                                                   
 def reverse_string(s):                                                                                            
     """                                                                                                           
     Reverses the given string.                                                                                    
                                                                                                                   
     Args:                                                                                                         
     s (str): The input string to be reversed                                                                      
                                                                                                                   
     Returns:                                                                                                      
     str: The reversed string                                                                                      
     """                                                                                                           
     # Method 1: Using string slicing                                                                              
     return s[::-1]                                                                                                
                                                                                                                   
 # Alternative implementations:                                                                                    
                                                                                                                   
 def reverse_string_method2(s):                                                                                    
     """                                                                                                           
     Reverses the string using a list reversal approach                                                            
     """                                                                                                           
     return ''.join(reversed(s))                                                                                   
                                                                                                                   
 def reverse_string_method3(s):                                                                                    
     """                                                                                                           
     Reverses the string using a manual iteration approach                                                         
     """                                                                                                           
     # Convert string to list of characters                                                                        
     chars = list(s)                                                                                               
                                                                                                                   
     # Two-pointer approach to swap characters                                                                     
     left = 0                                                                                                      
     right = len(chars) - 1                                                                                        
                                                                                                                   
     while left < right:                               

## 5. Conversation manager for multi-turn chats

In [19]:
conversation = []

import textwrap
from rich.console import Console
from rich.markdown import Markdown

console = Console()

def smart_print_markdown(output: str, width: int = 100):
    """
    Wrap plain text, preserve fenced code blocks.
    """
    in_code = False
    para_buf = []

    def flush_paragraph():
        if para_buf:
            text = " ".join(para_buf)
            print(textwrap.fill(text, width=width, replace_whitespace=False))
            print()
            para_buf.clear()

    for line in output.splitlines():
        fence = line.strip().startswith("```")
        if fence:
            # Finish any pending wrapped paragraph before toggling code
            flush_paragraph()
            print(line)
            in_code = not in_code
            continue

        if in_code:
            # Inside code block -> print verbatim
            print(line)
        else:
            # Outside code block -> buffer/wrap paragraphs
            if line.strip() == "":
                flush_paragraph()
            else:
                para_buf.append(line)

    flush_paragraph()

def chat_with_claude(
    prompt: str,
    system: str = "You are a helpful coding assistant.",
    render: str = "markdown",      # 'markdown' | 'wrapped' | 'none'
    return_text: bool = False,
    wrap_width: int = 100,
) -> str | None:
    """
    Send a prompt with conversation memory.
    - render='markdown'  -> pretty Markdown rendering (code blocks look great)
    - render='wrapped'   -> wrap only plain text, preserve code fences
    - render='none'      -> print nothing (use return_text=True if you need the string)
    """
    if not anthropic_key:
        raise RuntimeError("Missing ANTHROPIC_API_KEY.")

    conversation.append({"role": "user", "content": prompt})

    try:
        msg = client.messages.create(
            model=MODEL_NAME,
            max_tokens=1000,
            temperature=0.2,
            system=system,
            messages=conversation,
        )
        parts = [b.text for b in msg.content if getattr(b, "type", None) == "text"]
        output = "\n\n".join(parts).strip() or "(No text)"

        if render == "markdown":
            console.print(Markdown(output))
        elif render == "wrapped":
            smart_print_markdown(output, width=wrap_width)
        # render == 'none' -> no printing

        conversation.append({"role": "assistant", "content": output})
        return output if return_text else None

    except APIError as e:
        print("Anthropic API error:", e)
        raise

# Optional helpers
def reset_conversation():
    conversation.clear()

def last_reply() -> str | None:
    for m in reversed(conversation):
        if m["role"] == "assistant":
            return m["content"]
    return None


## 6. Example usage


## 🟢 `ask_claude` → One-off Q\&A

```python
ask_claude("Write a Python function that reverses a string.")
```

* Every call sends **just your new prompt** (and the system message) to Claude.
* Claude sees *no memory* of previous interactions.
* It’s like asking a single, isolated Stack Overflow question.

👉 Use this when you want **stateless, independent answers** — e.g., “give me code for X.”

---

## 🟡 `chat_with_claude` → Multi-turn conversation

```python
chat_with_claude("Explain recursion with an example.")
chat_with_claude("Now show me the same thing iteratively.")
```

* Keeps a global `conversation` list that stores **every message so far**.
* Each new call sends the **entire conversation history** to Claude.
* Claude can “remember” what you said before and respond in context.

👉 Use this for **follow-ups** where you don’t want to repeat context manually.

---

## 🔍 Side-by-side

| Function               | History? | Use case                                                 |
| ---------------------- | -------- | -------------------------------------------------------- |
| **ask\_claude**        | ❌ None   | One-shot Q\&A, quick snippets                            |
| **chat\_with\_claude** | ✅ Yes    | Back-and-forth exploration, tutoring, debugging sessions |

---

✅ Both create chats, but only `chat_with_claude` behaves like a *true ongoing dialogue*.



In [20]:
# Multi-turn example
chat_with_claude("Explain recursion with an example.")


I'll explain recursion using a classic example of calculating the factorial of a number.                           

Recursion is a programming technique where a function calls itself to solve a problem by breaking it down into     
smaller, more manageable subproblems.                                                                              

Here's an example in Python:                                                                                       

                                                                                                                   
 def factorial(n):                                                                                                 
     # Base case: factorial of 0 or 1 is 1                                                                         
     if n == 0 or n == 1:                                                                                          
         return 1                                                                                                  
                                                                                                                   
     # Recursive case: n! = n * (n-1)!                                                                             
     else:                                                                                                         
         return n * factorial(n - 1)                                                                               
                                                                                                                   
 # Example usage                                                                                                   
 print(factorial(5))  # Output: 120                                                                                
                                                                                                                   

Let's break down how this recursive factorial function works:                                                      

 1 Base Case:                                                                                                      
    • When n is 0 or 1, the function returns 1                                                                     
    • This is crucial to stop the recursion from continuing indefinitely                                           
 2 Recursive Case:                                                                                                 
    • For any number greater than 1, the function calls itself with a smaller number                               
    • 5! = 5 * 4!                                                                                                  
    • 4! = 4 * 3!                                                                                                  
    • And so on...                                                                                                 

Here's how the recursion unfolds for factorial(5):                                                                 

                                                                                                                   
 factorial(5)                                                                                                      
 = 5 * factorial(4)                                                                                                
 = 5 * (4 * factorial(3))                                                                                          
 = 5 * (4 * (3 * factorial(2)))                                                                                    
 = 5 * (4 * (3 * (2 * factorial(1))))                                                                              
 = 5 * (4 * (3 * (2 * 1)))                                                                                         
 = 5 * (4 * (3 * 2))                             

In [21]:
chat_with_claude("Now show me the same thing iteratively.")

I'll show you the iterative versions of the factorial and Fibonacci examples to contrast with the recursive        
approach:                                                                                                          

 1 Factorial (Iterative):                                                                                          

                                                                                                                   
 def factorial_iterative(n):                                                                                       
     # Handle edge cases                                                                                           
     if n < 0:                                                                                                     
         raise ValueError("Factorial is not defined for negative numbers")                                         
                                                                                                                   
     # Initialize result                                                                                           
     result = 1                                                                                                    
                                                                                                                   
     # Multiply numbers from 1 to n                                                                                
     for i in range(1, n + 1):                                                                                     
         result *= i                                                                                               
                                                                                                                   
     return result                                                                                                 
                                                                                                                   
 # Example usage                                                                                                   
 print(factorial_iterative(5))  # Output: 120                                                                      
                                                                                                                   

 2 Fibonacci (Iterative):                                                                                          

                                                                                                                   
 def fibonacci_iterative(n):                                                                                       
     # Handle edge cases                                                                                           
     if n < 0:                                                                                                     
         raise ValueError("Fibonacci is not defined for negative numbers")                                         
                                                                                                                   
     # Handle base cases                                                                                           
     if n <= 1:                                                                                                    
         return n                                                                                                  
                                                                                                                   
     # Initialize first two Fibonacci numbers                                                                      
     a, b = 0, 1                                                                                                   
                                                    


## 7. Shell commands in Colab

You can run terminal commands with `!` or `%%bash`:

```python
!echo "Hello from shell"
!python --version

%%bash
echo "Current directory: $(pwd)"
ls -lah | head -n 5
```
